# 1. Usando Django ORM, For e create

In [1]:
logfile = '../time_log.txt'

## Inserindo os dados com create.

In [2]:
import subprocess
import timeit

In [3]:
Shop.objects.all().delete()

(0, {'core.Shop': 0})

In [4]:
Shop.objects.create(
    name='Abel',
    data={"product": "limonada", "price": 7}
)

<Shop: Abel>

In [5]:
shops = [
    ('Anny', {"product": "lanche", "price": 24}),
    ('Jasmin', {"product": "notebook", "price": 1800, "online": True}),
    ('Michael', {"tires": 4, "price": 34}),
    ('Tifanny', {"course": "Python", "days": 100}),
]

In [6]:
def insert_data_with_create():
    tic1 = timeit.default_timer()
    for shop in shops:
        Shop.objects.create(
            name=shop[0],
            data=shop[1]
        )
    toc1 = timeit.default_timer()
    time1 = toc1 - tic1
    return round((time1), 3)

In [7]:
time1 = insert_data_with_create()

In [8]:
print(time1)

0.011


In [9]:
total_items = len(shops)
total_items

4

In [14]:
def timelog(total_items, _time, logfile, resource):
    total_items = f'{total_items:,}'.replace(',', '.')
    space = ' ' * (10 - len(total_items))
    time = round((_time), 3)
    subprocess.call(f"printf '{total_items} {space} -> {time}s\t --> Inserindo {total_items} registros com {resource}.\n' >> {logfile}", shell=True)

In [15]:
timelog(total_items, time1, logfile, 'Django create')

# 2. Usando Django ORM, For e bulk_create

In [16]:
Shop.objects.all().delete()

(5, {'core.Shop': 5})

In [17]:
def insert_data_with_bulk_create():
    tic2 = timeit.default_timer()
    aux = []
    for shop in shops:
        obj = Shop(name=shop[0], data=shop[1])
        aux.append(obj)
    Shop.objects.bulk_create(aux)
    toc2 = timeit.default_timer()
    time2 = toc2 - tic2
    return round((time2), 3)

In [18]:
time2 = insert_data_with_bulk_create()

In [19]:
print(time1)

0.011


In [20]:
print(time2)

0.004


In [21]:
print('Diferença', round((time1 / time2), 3))

Diferença 2.75


In [22]:
timelog(total_items, time2, logfile, 'Django bulk_create')

### Inserindo 10.000 registros com create.

In [29]:
import names
import time
import timeit
from random import randint
from functools import wraps
from memory_profiler import memory_usage

In [30]:
# https://hakibenita.com/fast-load-data-python-postgresql
def profile(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        fn_kwargs_str = ', '.join(f'{k}={v}' for k, v in kwargs.items())
        print(f'\n{fn.__name__}({fn_kwargs_str})')

        # Measure time
        t = time.perf_counter()
        retval = fn(*args, **kwargs)
        elapsed = time.perf_counter() - t
        print(f'Time   {elapsed:0.4}')

        # Measure memory
        mem, retval = memory_usage((fn, args, kwargs), retval=True, timeout=200, interval=1e-7)

        print(f'Memory {max(mem) - min(mem)}')
        return retval

    return inner

In [31]:
max_length = 10000

In [34]:
@profile
def gen_data(max_length):
    _items = []
    for item in range(max_length):
        title = names.get_last_name()
        quantity = randint(10,1000)
        _items.append((title, quantity))
    return _items

In [48]:
tic0 = timeit.default_timer()
items = gen_data(max_length)  # <--- Gerando os dados
toc0 = timeit.default_timer()
time0 = toc0 - tic0
round((time0), 3)
timelog(max_length, time0, logfile, 'Gerando com names')


gen_data()
Time   55.93
Memory 1.23828125


In [49]:
len(items)

10000

In [50]:
items[:5]

[('Coles', 377),
 ('Rios', 927),
 ('Lomax', 530),
 ('Cooley', 118),
 ('Bigelow', 674)]

In [51]:
def insert_data_with_create(items):
    tic1 = timeit.default_timer()
    for item in items:
        Product.objects.create(title=item[0], quantity=item[1])
    toc1 = timeit.default_timer()
    time1 = toc1 - tic1
    return round((time1), 3)

In [52]:
time1 = insert_data_with_create(items)

In [53]:
print(time1)

16.252


In [54]:
timelog(max_length, time1, logfile, 'Django create')

## Inserindo 10.000 registros com bulk_create.

In [55]:
def insert_data_with_bulk_create(items):
    aux_list = []
    tic2 = timeit.default_timer()
    for item in items:
        obj = Product(title=item[0], quantity=item[1])
        aux_list.append(obj)
    Product.objects.bulk_create(aux_list)
    toc2 = timeit.default_timer()
    time2 = toc2 - tic2
    return round((time2), 3)

In [56]:
time2 = insert_data_with_bulk_create(items)

In [57]:
print(time1)

16.252


In [58]:
print(time2)

0.433


In [59]:
print('Diferença', round(time1 / time2, 2))

Diferença 37.53


In [60]:
timelog(max_length, time2, logfile, 'Django bulk_create')

## Inserindo 100.000 registros com create.

In [61]:
max_length = 100000

In [62]:
'''
Não faça isso em casa, vai demorar 20 minutos e consumir o seu SSD.
# tic0 = timeit.default_timer()
# items = gen_data(max_length)  # <--- Gerando os dados
# toc0 = timeit.default_timer()
# time0 = toc0 - tic0
# round((time0), 3)
# timelog(max_length, time0, logfile, 'Gerando com names')
# Time   572.8  <-- na verdade é o dobro desse tempo por causa do profile.
# Memory 15.47265625
'''


gen_data()
Time   572.8
Memory 15.47265625


In [63]:
len(items)

100000

In [64]:
items[:5]

[('Passe', 562),
 ('West', 637),
 ('Schneider', 217),
 ('Perry', 879),
 ('Cordoua', 226)]

In [65]:
time1 = insert_data_with_create(items)

In [66]:
print(time1)

168.098


In [68]:
timelog(max_length, time1, logfile, 'Django create')

## Inserindo 100.000 registros com bulk_create.

In [69]:
time2 = insert_data_with_bulk_create(items)

In [70]:
print(time1)

168.098


In [71]:
print(time2)

4.866


In [72]:
print('Diferença', round(time1 / time2, 2))

Diferença 34.55


In [73]:
timelog(max_length, time2, logfile, 'Django bulk_create')

# Inserindo os dados de um CSV com bulk_create.

In [74]:
# Fechar o notebook ...
# E rodar insert_with_bulk_create.py